In [1]:
import pandas as pd
import polars as pl
import util
from pathlib import Path

pd.set_option('display.float_format', '{:,.0f}'.format)

## Average Miles Walking and Biking per Day by Resident

In [2]:
# trip,person,vmt data
person_trips = pd.read_csv(util.output_path / 'agg/dash/person_trips.csv')
person = pd.read_csv(util.output_path / 'agg/dash/person_geog.csv')
vmt = pd.read_csv(util.output_path / 'agg/dash/person_vmt.csv')

# list of equity geographies
equity_geogs = util.summary_config['hh_equity_geogs']

# TRIPS
df_trip = person_trips.copy()

# add home RGC
df_trip = util.get_is_rgc(df_trip, 'hh_rgc')
# add trip type
df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'

# PERSONS
df_person = person.copy()
# add home RGC
df_person = util.get_is_rgc(df_person, 'hh_rgc')

# VMT
df_vmt = vmt.copy()
# add home RGC
df_vmt = util.get_is_rgc(df_vmt, 'hh_rgc')

# Select only walk and bike trips
df_vmt_bp = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])].copy()
# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)].copy()

# total population by equity geography
equity_geogs_population = df_person[equity_geogs].apply(lambda x: x * df_person['psexpfac']).sum().reset_index()
equity_geogs_population.columns = ['Equity Group', 'psexpfac']

df_vmt_bp2 = df_vmt_bp.copy()
df_vmt_bp2['mode'] = 'Walk and Bike'
# add walk and bike
df_vmt_bp2 = pd.concat([df_vmt_bp, df_vmt_bp2])



In [3]:
def walk_bike_per_person(geog, map=False):
    
    # miles by mode and geography
    df1 = df_vmt_bp2.groupby([geog, 'mode'], as_index=False)['travdist_wt'].sum().set_index(geog)

    if map:
        df1.index = df1.index.astype('int').map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

    # add total miles in region
    df1_region = df_vmt_bp2.groupby(['mode'])['travdist_wt'].sum().reset_index()
    df1_region[geog] = 'Region'
    df1_region = df1_region.set_index(geog)
    df1 = pd.concat([df1, df1_region])

    # population by geography
    df3 = df_person.groupby(geog, as_index=False)['psexpfac'].sum().set_index(geog)
    if map:
        df3.index = df3.index.astype('int').map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })
    df3.loc['Region',:] = df3.sum(axis=0)

    # calculate average miles per person
    df = df1.merge(df3, on=geog)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']

    return df.pivot(columns='mode', values='Average Miles per Person')

In [4]:
pd.set_option('display.float_format', '{:,.2f}'.format)
walk_bike_per_person('hh_county')

mode,Bike,Walk,Walk and Bike
hh_county,,,
King,0.24,0.71,0.95
Kitsap,0.16,0.56,0.72
Outside Region,0.25,NaN,0.25
Pierce,0.20,0.57,0.77
Region,0.21,0.64,0.85
Snohomish,0.19,0.55,0.74


In [5]:
walk_bike_per_person('is_rgc')

mode,Bike,Walk,Walk and Bike
is_rgc,,,
In RGC,0.27,1.11,1.37
Not in RGC,0.20,0.55,0.75
Region,0.21,0.64,0.85


In [6]:
walk_bike_per_person('hh_rgc')

mode,Bike,Walk,Walk and Bike
hh_rgc,,,
Auburn,0.35,0.72,1.08
Bellevue,0.25,1.09,1.34
Bothell Canyon Park,0.27,0.59,0.85
Bremerton,0.19,0.92,1.11
Burien,0.31,0.64,0.96
Everett,0.19,0.97,1.16
Federal Way,0.31,0.74,1.05
Greater Downtown Kirkland,0.31,0.66,0.97
Issaquah,0.37,0.66,1.03


In [7]:
walk_bike_per_person('hh_rg_proposed')

mode,Bike,Walk,Walk and Bike
hh_rg_proposed,,,
Cities and Towns,0.16,0.45,0.61
Core Cities,0.23,0.61,0.84
High Capacity Transit Communities,0.20,0.53,0.74
Metropolitan Cities,0.24,0.90,1.14
Region,0.21,0.64,0.85
Rural Areas,0.17,0.32,0.48
Urban Unincorporated Areas,0.17,0.46,0.63


In [8]:
efa_col_dict = {
    "People of Color": "hh_efa_poc",
    "Income": "hh_efa_pov200",
    "LEP": "hh_efa_lep",
    "Disability": "hh_efa_dis",
    "Older Adults": "hh_efa_older",
    "Youth": "hh_efa_youth"
    }

In [9]:
df = pd.DataFrame()
for label, col in efa_col_dict.items():
    _df = walk_bike_per_person(col, map=True)
    _df = _df.drop('Region', axis=0)
    _df['Group'] = label
    df = pd.concat([df, _df])
df = df.reset_index()
df.rename(columns={'index': 'EFA Type'}, inplace=True)
df

mode,EFA Type,Bike,Walk,Walk and Bike,Group
0,Above Regional Average,0.22,0.71,0.93,People of Color
1,Below Regional Average,0.20,0.57,0.77,People of Color
2,Higher Share of Equity Population,0.24,0.70,0.95,People of Color
3,Above Regional Average,0.21,0.65,0.86,Income
4,Below Regional Average,0.21,0.59,0.80,Income
5,Higher Share of Equity Population,0.24,0.77,1.01,Income
6,Above Regional Average,0.23,0.66,0.89,LEP
7,Below Regional Average,0.20,0.62,0.82,LEP
8,Higher Share of Equity Population,0.24,0.66,0.90,LEP
9,Above Regional Average,0.21,0.62,0.83,Disability


## Median Trip Distance by Mode and Home Location 

In [10]:
trip = pl.read_csv(util.output_path / 'daysim/_trip.tsv',separator='\t')
hh = pl.read_csv(util.output_path / 'daysim/_household.tsv',separator='\t')

list_cols = ['ParcelID','CountyName','GrowthCenterName','rg_proposed', 'People of Color', 'Income', 'LEP', 'Disability',
       'Older Adults', 'Youth']
parcel_geog = util.get_parcel_geog()[list_cols]


In [11]:
# trip table with home location
df_trip_hh = trip.join(hh, on="hhno", how="left").to_pandas()
df_trip_hh = df_trip_hh.merge(parcel_geog, how='left', left_on='hhparcel', right_on='ParcelID')
df_trip_hh['mode_label'] = df_trip_hh['mode'].map(
    {1: 'Walk',
     2: 'Bike',
     3: 'Drive Alone',
     4: 'Shared Ride',
     5: 'Shared Ride',
     6: 'Transit'}
     )

df_trip_hh = df_trip_hh[df_trip_hh['CountyName']!='Outside Region']
df_trip_hh = util.get_is_rgc(df_trip_hh, 'GrowthCenterName', 'is_rgc')


In [12]:
def get_median_time_distance(df, geog, value_col):

    df_geog = pd.pivot_table(df, index=geog, columns='mode_label', aggfunc='median', values=value_col)
    
    return df_geog


df = get_median_time_distance(df_trip_hh,'CountyName','travdist')
df_region =  get_median_time_distance(df_trip_hh,None,'travdist')
df.loc['Region'] = df_region.iloc[0]
df

mode_label,Bike,Drive Alone,Shared Ride,Transit,Walk
CountyName,,,,,
King,1.48,3.60,2.60,4.49,0.52
Kitsap,1.59,2.92,2.31,11.72,0.62
Pierce,1.65,3.70,2.79,3.73,0.61
Snohomish,1.71,3.53,2.52,6.27,0.67
Region,1.56,3.55,2.60,4.75,0.56


In [13]:
df = get_median_time_distance(df_trip_hh,'rg_proposed','travdist')
df

mode_label,Bike,Drive Alone,Shared Ride,Transit,Walk
rg_proposed,,,,,
Cities and Towns,1.65,4.58,2.84,10.46,0.72
Core Cities,1.72,3.32,2.43,7.24,0.63
High Capacity Transit Communities,1.94,3.53,2.61,8.30,0.79
Metropolitan Cities,1.22,2.49,1.85,3.42,0.47
Rural Areas,3.02,6.54,5.60,13.42,0.89
Urban Unincorporated Areas,2.21,4.19,3.42,9.76,1.01


In [14]:
df = get_median_time_distance(df_trip_hh,'is_rgc','travdist')
df

mode_label,Bike,Drive Alone,Shared Ride,Transit,Walk
is_rgc,,,,,
In RGC,0.95,1.85,1.26,2.21,0.39
Not in RGC,1.90,3.69,2.74,5.71,0.76


In [15]:
df = get_median_time_distance(df_trip_hh,'GrowthCenterName','travdist')
df

mode_label,Bike,Drive Alone,Shared Ride,Transit,Walk
GrowthCenterName,,,,,
Auburn,1.27,1.90,1.29,8.96,0.34
Bellevue,0.71,1.09,0.88,2.67,0.32
Bothell Canyon Park,1.85,3.48,2.70,8.18,0.55
Bremerton,0.77,0.91,0.70,9.84,0.29
Burien,1.13,3.91,1.43,8.72,0.32
Everett,0.72,0.86,0.70,1.10,0.34
Federal Way,1.13,2.26,1.38,7.51,0.38
Greater Downtown Kirkland,2.10,2.77,1.96,9.29,0.47
Issaquah,1.30,3.24,1.51,9.73,0.38


In [16]:
df = pd.DataFrame()
for col in efa_col_dict.keys():
    _df = get_median_time_distance(df_trip_hh,col,'travdist').reset_index()
    _df['Group'] = col
    _df.rename(columns={col: 'EFA Type'}, inplace=True)
    df = pd.concat([df, _df])

df.set_index(['Group','EFA Type'])

mode_label                                         Bike  Drive Alone  \
Group           EFA Type                                               
People of Color Above Regional Average             1.43         3.30   
                Below Regional Average             1.71         3.79   
                Higher Share of Equity Population  1.48         3.36   
Income          Above Regional Average             1.53         3.21   
                Below Regional Average             1.67         3.94   
                Higher Share of Equity Population  1.34         2.71   
LEP             Above Regional Average             1.56         3.60   
                Below Regional Average             1.54         3.65   
                Higher Share of Equity Population  1.60         3.20   
Disability      Above Regional Average             1.60         3.43   
                Below Regional Average             1.66         3.80   
                Higher Share of Equity Population  1.29         2.91   
Older Adults    Above Regional Average             1.62         3.76   
                Below Regional Average             1.51         3.39   
                Higher Share of Equity Population  1.67         3.79   
Youth           Above Regional Average             1.90         3.88   
                Below Regional Average             1.38         3.11   
                Higher Share of Equity Population  1.97         4.49   

mode_label                                         Shared Ride  Transit  Walk  
Group           EFA Type                                                       
People of Color Above Regional Average                    2.40     4.09  0.52  
                Below Regional Average                    2.79     5.29  0.63  
                Higher Share of Equity Population         2.48     4.56  0.53  
Income          Above Regional Average                    2.34     4.74  0.57  
                Below Regional Average                    2.92     5.07  0.60  
                Higher Share of Equity Population         2.02     4.11  0.50  
LEP             Above Regional Average                    2.67     5.24  0.53  
                Below Regional Average                    2.67     4.50  0.56  
                Higher Share of Equity Population         2.34     5.14  0.60  
Disability      Above Regional Average                    2.55     5.24  0.60  
                Below Regional Average                    2.80     4.83  0.60  
                Higher Share of Equity Population         2.06     3.08  0.48  
Older Adults    Above Regional Average                    2.76     5.18  0.59  
                Below Regional Average                    2.47     4.30  0.55  
                Higher Share of Equity Population         2.82     6.78  0.55  
Youth           Above Regional Average                    2.89     6.57  0.74  
                Below Regional Average                    2.26     4.18  0.50  
                Higher Share of Equity Population         3.18     7.80  0.83

## Median Time Spent Walking/Biking
Time in minutes

In [17]:

df = get_median_time_distance(df_trip_hh,'CountyName','travtime')
df_region =  get_median_time_distance(df_trip_hh,None,'travtime')
df.loc['Region'] = df_region.iloc[0]
df

mode_label,Bike,Drive Alone,Shared Ride,Transit,Walk
CountyName,,,,,
King,9.84,12.38,10.31,23.67,10.47
Kitsap,10.60,10.36,9.28,65.08,12.33
Pierce,10.97,12.39,10.68,28.48,12.17
Snohomish,11.38,11.38,9.34,30.62,13.33
Region,10.39,12.03,10.12,25.39,11.25


In [18]:
df = get_median_time_distance(df_trip_hh,'rg_proposed','travtime')
df

mode_label,Bike,Drive Alone,Shared Ride,Transit,Walk
rg_proposed,,,,,
Cities and Towns,11.02,12.79,9.77,48.54,14.43
Core Cities,11.46,11.16,9.38,34.42,12.64
High Capacity Transit Communities,12.92,11.50,9.64,34.95,15.71
Metropolitan Cities,8.13,11.40,9.83,19.84,9.42
Rural Areas,20.13,16.30,14.90,60.80,17.77
Urban Unincorporated Areas,14.73,12.44,11.09,44.97,20.20


In [19]:
df = get_median_time_distance(df_trip_hh,'is_rgc','travtime')
df

mode_label,Bike,Drive Alone,Shared Ride,Transit,Walk
is_rgc,,,,,
In RGC,6.36,10.40,8.93,15.65,7.87
Not in RGC,12.64,12.20,10.25,29.07,15.10


In [20]:
df = get_median_time_distance(df_trip_hh,'GrowthCenterName','travtime')
df

mode_label,Bike,Drive Alone,Shared Ride,Transit,Walk
GrowthCenterName,,,,,
Auburn,8.49,9.11,7.87,37.08,6.80
Bellevue,4.73,7.82,7.03,19.17,6.33
Bothell Canyon Park,12.35,11.82,9.92,43.69,10.99
Bremerton,5.10,8.83,8.19,43.66,5.90
Burien,7.52,11.15,7.11,36.56,6.47
Everett,4.80,6.98,6.31,13.24,6.88
Federal Way,7.54,9.08,7.08,27.87,7.58
Greater Downtown Kirkland,13.99,9.98,8.50,34.05,9.48
Issaquah,8.67,10.54,6.26,37.10,7.64


In [21]:
df = pd.DataFrame()
for col in efa_col_dict.keys():
    _df = get_median_time_distance(df_trip_hh,col,'travtime').reset_index()
    _df['Group'] = col
    _df.rename(columns={col: 'EFA Type'}, inplace=True)
    df = pd.concat([df, _df])

df.set_index(['Group','EFA Type'])

mode_label                                         Bike  Drive Alone  \
Group           EFA Type                                               
People of Color Above Regional Average             9.55        11.52   
                Below Regional Average            11.37        12.49   
                Higher Share of Equity Population  9.89        11.64   
Income          Above Regional Average            10.19        11.41   
                Below Regional Average            11.17        12.65   
                Higher Share of Equity Population  8.93        10.79   
LEP             Above Regional Average            10.41        11.86   
                Below Regional Average            10.29        12.37   
                Higher Share of Equity Population 10.69        11.19   
Disability      Above Regional Average            10.67        11.80   
                Below Regional Average            11.08        12.45   
                Higher Share of Equity Population  8.59        11.07   
Older Adults    Above Regional Average            10.82        12.38   
                Below Regional Average            10.06        11.74   
                Higher Share of Equity Population 11.10        12.34   
Youth           Above Regional Average            12.67        12.26   
                Below Regional Average             9.22        11.64   
                Higher Share of Equity Population 13.13        13.03   

mode_label                                         Shared Ride  Transit  Walk  
Group           EFA Type                                                       
People of Color Above Regional Average                    9.71    23.50 10.34  
                Below Regional Average                   10.52    26.42 12.53  
                Higher Share of Equity Population         9.80    25.74 10.64  
Income          Above Regional Average                    9.64    25.43 11.35  
                Below Regional Average                   10.62    26.47 11.91  
                Higher Share of Equity Population         9.29    22.20  9.98  
LEP             Above Regional Average                    9.97    26.61 10.69  
                Below Regional Average                   10.46    24.10 11.28  
                Higher Share of Equity Population         9.34    27.66 11.97  
Disability      Above Regional Average                    9.99    26.32 11.90  
                Below Regional Average                   10.43    25.82 11.92  
                Higher Share of Equity Population         9.43    21.33  9.59  
Older Adults    Above Regional Average                   10.45    27.86 11.74  
                Below Regional Average                    9.83    23.43 11.06  
                Higher Share of Equity Population        10.58    29.32 11.08  
Youth           Above Regional Average                   10.27    33.21 14.86  
                Below Regional Average                    9.87    22.61 10.05  
                Higher Share of Equity Population        10.67    38.84 16.67

## % Population Walking or Biking for Transportation
Does not include people making exercise trips

In [22]:
bike_walk_trips = trip.filter(pl.col("mode").is_in([1, 2]))

# Get unique persons with at least one bike/walk trip
bike_walk_persons = bike_walk_trips.select(["hhno", "pno"]).unique()
bike_walk_persons = bike_walk_persons.with_columns(bike_walk=pl.lit(1))

# Join back to all persons, mark bike_walk as False if not present
person = pl.read_csv(util.output_path / 'daysim/_person.tsv',separator='\t')
person_with_bike_walk = person.join(bike_walk_persons, on=["hhno", "pno"], how="left")
person_with_bike_walk = person_with_bike_walk.join(hh, on='hhno', how='left')

person_with_bike_walk = person_with_bike_walk.to_pandas()
person_with_bike_walk = person_with_bike_walk.merge(parcel_geog, how='left', left_on='hhparcel', right_on='ParcelID')

person_with_bike_walk['bike_walk'] = person_with_bike_walk['bike_walk'].fillna(0)


person_with_bike_walk = person_with_bike_walk[person_with_bike_walk['CountyName']!='Outside Region']
person_with_bike_walk = util.get_is_rgc(person_with_bike_walk, 'GrowthCenterName', 'is_rgc')

In [23]:
def get_pop_bike_walk(df, geog):
    df = pd.pivot_table(df, index=geog, columns='bike_walk', aggfunc='sum', values='psexpfac')
    df['% Biking or Walking'] = df[1]/(df[0]+df[1])
    return df[['% Biking or Walking']]

In [24]:
pd.set_option('display.float_format', '{:.1%}'.format)
df = get_pop_bike_walk(person_with_bike_walk, 'CountyName')
df_region =  get_pop_bike_walk(person_with_bike_walk, None)
df.loc['Region'] = df_region.iloc[0]
df

bike_walk,% Biking or Walking
CountyName,
King,36.6%
Kitsap,28.6%
Pierce,27.5%
Snohomish,27.0%
Region,32.2%


In [25]:
df = get_pop_bike_walk(person_with_bike_walk, 'rg_proposed')
df

bike_walk,% Biking or Walking
rg_proposed,
Cities and Towns,23.2%
Core Cities,30.7%
High Capacity Transit Communities,25.2%
Metropolitan Cities,48.3%
Rural Areas,13.2%
Urban Unincorporated Areas,18.4%


In [26]:
df = get_pop_bike_walk(person_with_bike_walk, 'is_rgc')
df

bike_walk,% Biking or Walking
is_rgc,
In RGC,64.3%
Not in RGC,26.2%


In [27]:
df = get_pop_bike_walk(person_with_bike_walk, 'GrowthCenterName')
df

bike_walk,% Biking or Walking
GrowthCenterName,
Auburn,51.2%
Bellevue,74.5%
Bothell Canyon Park,31.2%
Bremerton,68.0%
Burien,48.4%
Everett,67.0%
Federal Way,50.1%
Greater Downtown Kirkland,39.9%
Issaquah,47.8%


In [28]:
df = pd.DataFrame()
for col in efa_col_dict.keys():
    _df = get_pop_bike_walk(person_with_bike_walk,col).reset_index()
    _df['Group'] = col
    _df.rename(columns={col: 'EFA Type'}, inplace=True)
    df = pd.concat([df, _df])

df.set_index(['Group','EFA Type'])

bike_walk                                          % Biking or Walking
Group           EFA Type                                              
People of Color Above Regional Average                           36.3%
                Below Regional Average                           28.4%
                Higher Share of Equity Population                35.8%
Income          Above Regional Average                           33.1%
                Below Regional Average                           29.2%
                Higher Share of Equity Population                40.7%
LEP             Above Regional Average                           33.3%
                Below Regional Average                           31.3%
                Higher Share of Equity Population                33.5%
Disability      Above Regional Average                           30.9%
                Below Regional Average                           30.4%
                Higher Share of Equity Population                39.4%
Older Adults    Above Regional Average                           29.3%
                Below Regional Average                           34.3%
                Higher Share of Equity Population                30.5%
Youth           Above Regional Average                           24.7%
                Below Regional Average                           39.0%
                Higher Share of Equity Population                21.6%

## Bike and Walk Tours by Purpose
Tours where most or all trips are by the mode.

In [29]:
df = pd.read_csv(util.output_path / 'agg/dash/tour_total.csv')

df['Tour Mode'] = df['tmodetp'].map({'SOV': 'Drove Alone',
                                     'HOV2': 'Shared Ride',
                                     'HOV3+': 'Shared Ride',
                                     'Transit': 'Transit',
                                     'Walk': 'Walk',
                                     'Bike': 'Bike'
                                     })

df = pd.pivot_table(df, index='pdpurp', columns='Tour Mode', values='toexpfac', aggfunc='sum')
df = df/df.sum()
df[['Transit', 'Walk', 'Bike','Drove Alone', 'Shared Ride', ]]

Tour Mode,Transit,Walk,Bike,Drove Alone,Shared Ride
pdpurp,,,,,
Escort,0.0%,8.4%,3.4%,1.0%,22.9%
Meal,8.0%,10.6%,1.8%,5.0%,8.0%
Personal Business,11.9%,5.5%,3.9%,8.2%,5.8%
School,17.9%,4.2%,16.0%,3.5%,10.6%
Shop,16.9%,12.7%,7.9%,14.8%,11.0%
Social,19.0%,43.3%,37.2%,21.6%,23.3%
Work,26.2%,15.4%,29.8%,45.9%,18.4%


## Average Jobs Accessible within 1 Mile Walk and 3 Mile Bike
Note that this is not using the bike network, but is instead using the all-streets network.

Average accessible jobs are weighted averages based on parcel household population.

In [30]:
# total jobs
parcel_emp = util.get_parcels_urbansim_data()[['parcelid','emptot_p']]
tot_jobs = parcel_emp['emptot_p'].sum()

In [31]:
walk_bike_jobs_access = pd.read_csv(util.output_path / 'access/walk_bike_jobs_access.csv', 
                                  usecols=['geography_value', 'jobs_1_mile_walk', 'jobs_3_mile_bike', 'geography_group']).\
                                  rename(columns={'geography_value': 'geography'})

df_access = walk_bike_jobs_access.copy()
df_access = df_access[df_access['geography'] != 'Outside Region']
# rename region
df_access.loc[df_access['geography_group'] == 'region', 'geography'] = 'Region'
# rename rgc
df_access.loc[df_access['geography_group'] == 'rgc_binary', 'geography'] = ['Not in RGC', 'In RGC']
# rename efa 
# jobs access in equity geographies
equity_geogs = util.summary_config['equity_geogs']
df_access.loc[df_access['geography_group'].isin(equity_geogs), 'geography'] = df_access.loc[df_access['geography_group'].isin(equity_geogs), 'geography'].\
    map({"0.0": 'Below Regional Average', 
         "1.0": 'Above Regional Average', 
         "2.0": 'Higher Share of Equity Population'}
         )

In [32]:
def bp_job_access_geog(access_table,geog):
    df = access_table.loc[access_table['geography_group'].isin(geog)].\
        rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}).\
        drop(columns=['geography_group']).\
        set_index('geography')

    df['% Total Jobs (1-mile Walk)'] = df['Jobs within 1-mile Walk'].apply(lambda x: x / tot_jobs)
    df['% Total Jobs (3-mile Bike)'] = df['Jobs within 3-mile Bike'].apply(lambda x: x / tot_jobs)

    

    return df

format_dict = {
        'Jobs within 1-mile Walk': "{:.0f}",
        'Jobs within 3-mile Bike': "{:.0f}", 
        '% Total Jobs (1-mile Walk)': "{:.1%}", 
        '% Total Jobs (3-mile Bike)': "{:.1%}"
    }

In [33]:
df = bp_job_access_geog(df_access,['region','CountyName'])
df.style.format(format_dict)

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile Walk),% Total Jobs (3-mile Bike)
geography,,,,
King,37271,134135,1.2%,4.2%
Kitsap,2746,14528,0.1%,0.5%
Pierce,8283,37911,0.3%,1.2%
Snohomish,6443,36812,0.2%,1.2%
Region,22998,87445,0.7%,2.8%


In [34]:
df = bp_job_access_geog(df_access,['rgc_binary'])
df.style.format(format_dict)

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile Walk),% Total Jobs (3-mile Bike)
geography,,,,
Not in RGC,4676,49794,0.1%,1.6%
In RGC,104176,254260,3.3%,8.0%


In [35]:
df = bp_job_access_geog(df_access,['GrowthCenterName'])
df.style.format(format_dict)

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile Walk),% Total Jobs (3-mile Bike)
geography,,,,
Auburn,19999,58805,0.6%,1.9%
Bellevue,105531,180124,3.3%,5.7%
Bothell Canyon Park,13092,34742,0.4%,1.1%
Bremerton,20449,48459,0.6%,1.5%
Burien,10870,22675,0.3%,0.7%
Everett,48673,91292,1.5%,2.9%
Federal Way,20181,50482,0.6%,1.6%
Greater Downtown Kirkland,20518,58731,0.6%,1.9%
Issaquah,11322,31886,0.4%,1.0%


In [36]:
df = bp_job_access_geog(df_access,['rg_proposed'])
df.style.format(format_dict)

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile Walk),% Total Jobs (3-mile Bike)
geography,,,,
Cities and Towns,1383,12707,0.0%,0.4%
Core Cities,8355,49234,0.3%,1.6%
High Capacity Transit Communities,2793,26296,0.1%,0.8%
Metropolitan Cities,57573,194889,1.8%,6.2%
Rural Areas,243,3539,0.0%,0.1%
Urban Unincorporated Areas,685,10604,0.0%,0.3%


In [37]:
efa_geog_dict = {
    "People of Color": "equity_focus_areas_2023__efa_poc",
    "Income": "equity_focus_areas_2023__efa_pov200",
    "LEP": "equity_focus_areas_2023__efa_lep",
    "Disability": "equity_focus_areas_2023__efa_dis",
    "Older Adults": "equity_focus_areas_2023__efa_older",
    "Youth": "equity_focus_areas_2023__efa_youth"
    }

df = pd.DataFrame()
for label, col in efa_geog_dict.items():
    _df = bp_job_access_geog(df_access,[col])
    _df['Group'] = label
    df = pd.concat([df, _df])
df = df.reset_index()
df.rename(columns={'geography': 'EFA Type'}, inplace=True)
df.set_index(['Group','EFA Type']).style.format(format_dict)